In [ ]:
!pip install mediapipe opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
from google.colab.patches import cv2_imshow  # For showing frames in Colab


In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)  # Initialize with desired parameters


In [ ]:
# Load the video file (replace with the path to your video in Colab)
video_path = '/content/e020.mp4'  # Upload the video to Colab or link from drive
cap = cv2.VideoCapture(video_path)

In [ ]:
# Get the frame rate (frames per second) of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"FPS of the video: {fps}")

FPS of the video: 30


In [ ]:
# Set the time duration to process (in seconds)
duration = 10  # Process for 10 seconds
max_frames = duration * fps  # Calculate the number of frames to process

In [ ]:
# Prepare to save landmark pixel coordinates
landmark_data = []
frame_count = 0

In [ ]:
# Process video frame by frame
while cap.isOpened() and frame_count < max_frames: # Use max_frames instead of total_frames
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    height, width, _ = frame.shape

    # Convert the frame to RGB as MediaPipe requires
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand landmark detection
    result = hands.process(rgb_frame)

    # Check if landmarks were detected
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            for idx, landmark in enumerate(hand_landmarks.landmark):
                # Convert normalized coordinates to pixel coordinates
                x_pixel = int(landmark.x * width)
                y_pixel = int(landmark.y * height)

                # Record frame number, landmark index, x, and y pixel coordinates
                landmark_data.append([frame_count, idx, x_pixel, y_pixel])

            # Optional: Draw landmarks on the frame for visualization (if needed)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Increment the frame count
    frame_count += 1

# Release resources
cap.release()
hands.close()

In [ ]:
# Save the landmark data to a CSV file automatically
df = pd.DataFrame(landmark_data, columns=['Frame', 'Landmark_Index', 'X_coordinate', 'Y_coordinate'])
df.to_csv('/content/e020.csv', index=False)

print("Hand landmark pixel coordinates saved to 'e020.csv'")

Hand landmark pixel coordinates saved to 'e020.csv'
